In [ ]:
#import data from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import string
import json
import warnings
warnings.filterwarnings(action = 'ignore')
import itertools
import numpy as np
import math
from numpy import dot
from numpy.linalg import norm

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

!pip install datasets
from datasets import Dataset

!pip install --upgrade openai
import openai

**Train ML Model--Contributor**

In [ ]:
##### read labeled sentences
df_train = pd.read_excel("\sample_data.xlsx")

for var in ['pa','con_det',	'sp_non']:
    df_train [var] = df_train [var].fillna(-999)
    df_train [var] = df_train [var].astype(int)

df_train_pa = df_train[df_train['pa'] != -999]
df_train_pa = df_train_pa[['sentence','pa']]

df_train_con_det = df_train[df_train['con_det'] != -999]
df_train_con_det = df_train_con_det[['sentence','con_det']]
df_train_con_det.reset_index(drop=True, inplace=True)

df_train_sp_non = df_train[df_train['sp_non'] != -999]
df_train_sp_non = df_train_sp_non[['sentence','sp_non']]
df_train_sp_non.reset_index(drop=True, inplace=True)

In [ ]:
##### read labeled sentences
df_train = pd.read_excel(".xlsx")

for var in ['pa','con_det',	'sp_non']:
    df_train [var] = df_train [var].fillna(-999)
    df_train [var] = df_train [var].astype(int)

df_train_pa = df_train[df_train['pa'] != -999]
df_train_pa = df_train_pa[['sentence','pa']]

df_train_con_det = df_train[df_train['con_det'] != -999]
df_train_con_det = df_train_con_det[['sentence','con_det']]
df_train_con_det.reset_index(drop=True, inplace=True)

df_train_sp_non = df_train[df_train['sp_non'] != -999]
df_train_sp_non = df_train_sp_non[df_train_sp_non['con_det'] == 1] ################################ conditional ############################
df_train_sp_non = df_train_sp_non[['sentence','sp_non']]
df_train_sp_non.reset_index(drop=True, inplace=True)

df_train = pd.read_excel(".xlsx")

for var in ['pa','con_det',	'sp_non']:
    df_train [var] = df_train [var].fillna(-999)
    df_train [var] = df_train [var].astype(int)

df_train['pa'] = 0
df_train.loc[df_train.con_det > -999, 'pa'] = 1
df_train_pa2 = df_train[df_train['pa'] != -999]
df_train_pa2 = df_train_pa2[['sentence','pa']]

df_train_con_det2 = df_train[df_train['con_det'] != -999]
df_train_con_det2 = df_train_con_det2[['sentence','con_det']]
df_train_con_det2.reset_index(drop=True, inplace=True)

df_train_sp_non2 = df_train[df_train['sp_non'] != -999]
df_train_sp_non2 = df_train_sp_non2[df_train_sp_non2['con_det'] == 1] ################################ conditional ############################
df_train_sp_non2 = df_train_sp_non2[['sentence','sp_non']]
df_train_sp_non2.reset_index(drop=True, inplace=True)

df_train_pa = df_train_pa.append(df_train_pa2)
df_train_con_det = df_train_con_det.append(df_train_con_det2)
df_train_sp_non = df_train_sp_non.append(df_train_sp_non2)


df_train_pa.reset_index(drop=True, inplace=True)
df_train_con_det.reset_index(drop=True, inplace=True)
df_train_sp_non.reset_index(drop=True, inplace=True)

'''###################### contributor labels ######################'''
# form a training set--contributor/detractor
df_train2 = Dataset.from_pandas(df_train_con_det)
df_train2 = df_train2.rename_column('con_det','label')
df_train2

'''###################### bert ######################'''
Tokenizer_name = "bert-base-uncased"                       ################################ model selection ############################
Model_name = "bert-base-uncased"                           ################################ model selection ############################
Number_of_labels = 2

!pip install transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(Tokenizer_name)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = df_train2.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(['sentence'])



#Stratified k-fold sampling and modeling
label_for_s_kfold = tokenized_datasets['label']
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

from transformers import TrainingArguments

from datasets import load_metric
training_args = TrainingArguments("test_trainer")

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import Trainer


s_kfold = StratifiedKFold(n_splits=5, random_state=None, shuffle=False) #set k =5
my_accuracy = []
my_precison = []
my_recall = []
my_fscore = []
my_confusion_matrix = []

for train_index, test_index in s_kfold.split(np.zeros(len(label_for_s_kfold)), label_for_s_kfold):
  my_train_set = tokenized_datasets.select(train_index)
  my_test_set = tokenized_datasets.select(test_index )

  model = AutoModelForSequenceClassification.from_pretrained(Model_name, num_labels = Number_of_labels)

  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=my_train_set,
    eval_dataset=my_test_set,
    compute_metrics=compute_metrics,
)
  trainer.train()

  predictions = trainer.predict(my_test_set)
  array_predict = predictions.predictions
  list_actual = my_test_set['label']
  list_predict = []
  for i in range(array_predict.shape[0]):
    list_predict.append(np.argmax(array_predict[i]))

  my_precison_new, my_recall_new, my_fscore_new, my_support_new = precision_recall_fscore_support(list_actual, list_predict, average='weighted')

  my_accuracy.append(trainer.evaluate()['eval_accuracy'])

  my_precison.append(my_precison_new)
  my_recall.append(my_recall_new)
  my_fscore.append(my_fscore_new)

  my_confusion_matrix.append(confusion_matrix(list_actual,list_predict))

print("fscore:", sum(my_fscore)/len(my_fscore))
print("confusion_matrix:\n", sum(my_confusion_matrix) / len(my_confusion_matrix))






'''###################### get the relevant sentence ######################'''
'''load sentence count'''
dfn = pd.read_csv(".csv")
dfn.reset_index(drop=True, inplace=True)

'''contrsuct sentence corpus'''
dfn_original = pd.read_csv(".csv")

repeat_times = []
mycorpus_sent = []

for index,row in dfn_original.iterrows():
    repeat_times.append(len(sent_tokenize(dfn_original['discussion'][index])))
    for my_sent in sent_tokenize(dfn_original['discussion'][index]):

        mycorpus_sent.append(my_sent)

dfn_sentence = dfn_original[['cik','date_filing']]
dfn_sentence = dfn_sentence.loc[dfn_original.index.repeat(repeat_times)]

dfn_sentence['sentence'] = mycorpus_sent
dfn_sentence.reset_index(drop=True, inplace=True)



'''###################### sentence prediction ######################'''
dfn_perfprmance_index = dfn.index[dfn.drop(['cik','date_filing','total_word'], axis=1).sum(axis = 1) > 0]
dfn_performance = dfn_sentence[dfn_sentence.index.isin(dfn_perfprmance_index)]

dfn_performance_2 = dfn_performance[['cik','date_filing']]
df_test1 = dfn_performance[['sentence']]
df_test2 = Dataset.from_pandas(df_test1)


tokenized_datasets = df_test2.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['sentence'])
my_test_set = tokenized_datasets

# predict using NLP model
predictions = trainer.predict(my_test_set)
array_predict = predictions.predictions

list_predict = []
for i in range(array_predict.shape[0]):
  list_predict.append(np.argmax(array_predict[i]))

dfn_performance_2["con_det"] = list_predict


#save the results
dfn_performance_2.reset_index(inplace=True)
dfn_performance_2 = dfn_performance_2.rename(columns = {'index':'orginal_index'})
dfn_performance_2.to_csv(".csv",index = False)


**GPT3**

In [ ]:
'''###################### get the relevant sentence ######################'''
'''load sentence count'''
dfn = pd.read_csv(".csv")
dfn.reset_index(drop=True, inplace=True)

'''contrsuct sentence corpus'''
dfn_original = pd.read_csv(".csv")

repeat_times = []
mycorpus_sent = []

for index,row in dfn_original.iterrows():
    repeat_times.append(len(sent_tokenize(dfn_original['discussion'][index])))
    for my_sent in sent_tokenize(dfn_original['discussion'][index]):

        mycorpus_sent.append(my_sent)

dfn_sentence = dfn_original[['cik','date_filing']]
dfn_sentence = dfn_sentence.loc[dfn_original.index.repeat(repeat_times)]

dfn_sentence['sentence'] = mycorpus_sent
dfn_sentence.reset_index(drop=True, inplace=True)

'''###################### contrcut predict set ######################'''
dfn_perfprmance_index = dfn.index[dfn.drop(['cik','date_filing','total_word'], axis=1).sum(axis = 1) > 0]
dfn_performance = dfn_sentence[dfn_sentence.index.isin(dfn_perfprmance_index)]

dfn_performance_2 = dfn_performance[['cik','date_filing']]
df_test = dfn_performance[['sentence']]
df_test.reset_index(inplace=True)

dfn_performance_2.reset_index(inplace=True)
dfn_performance_2 = dfn_performance_2.rename(columns = {'index':'orginal_index'})

In [ ]:
'''###################### choose model ######################'''
openai.api_key =""

ft_model = ''

In [ ]:
'''###################### sentence prediction ######################'''
div_obs = 5000
last_part_n_saw = 10

for i in range(last_part_n_saw, df_test.shape[0]//div_obs + 1):
  # index & corpus subsample
  dfn_performance_2_sub = dfn_performance_2[i*div_obs:(i+1)*div_obs]
  dfn_performance_2_sub.reset_index(drop=True, inplace=True)

  df_test_sub = df_test[i*div_obs:(i+1)*div_obs]
  df_test_sub.reset_index(drop=True, inplace=True)

  list_predict = []
  for index,row in df_test_sub.iterrows():
    my_prompt = df_test_sub['sentence'][index] + " ->"

    res = openai.Completion.create(model=ft_model, prompt=my_prompt, max_tokens=1, temperature=0)

    list_predict.append(res['choices'][0]['text'])

  dfn_performance_2_sub["con_det"] = list_predict
  dfn_performance_2_sub.to_csv(""+str(i + 1)+".csv",index = False)


In [ ]:
#append all parts
df_train_sub = pd.read_csv("")
for i in range(1, dfn_performance_2.shape[0]//div_obs + 1):
  df_train_sub_next = pd.read_csv(""+str(i + 1)+".csv")
  df_train_sub = df_train_sub.append(df_train_sub_next)

df_train_sub.to_csv(".csv",index = False)